In [ ]:
# convert source data into json-ld.

import ansible_vault
import datetime
import json
import pathlib
import pydash
import rdflib
import uuid

def statement_wrapper(s, p, o, x, k):

    """Wrap statements as literals, primarily for individual encryption."""

    meta_graph = rdflib.Graph()
    meta_graph.bind("ont", ont)
    meta_graph.bind("res", res)

    statement_id = res[str(uuid.uuid4())]

    if x == "open":
        meta_graph.add((statement_id, rdflib.RDF.type, ont.OpenStatement))
    elif x == "closed":
        meta_graph.add((statement_id, rdflib.RDF.type, ont.ClosedStatement))
        k[str(statement_id)] = str(uuid.uuid4())
    else:
        raise Exception("Statements must be either open or closed!")

    mini_graph = rdflib.Graph()
    mini_graph.bind("ont", ont)
    mini_graph.bind("res", res)
    mini_graph.add((s, p, o))

    payload = rdflib.Literal(str(mini_graph.serialize(format="n3")))
    meta_graph.add((statement_id, ont.hasPayload, payload))

    return meta_graph

with open(pathlib.Path.cwd().parents[0] / 'source' / 'birth_cert.json') as my_birth:
    my_birth = json.load(my_birth)

key_chain = dict()

ont = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/")
res = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/")

surface = rdflib.Graph()
surface.bind("ont", ont)
surface.bind("res", res)

surface += statement_wrapper(
    res[pydash.get(my_birth, "id")],
    rdflib.RDF.type,
    ont[pydash.get(my_birth, "event_type")],
    "open",
    key_chain,
)
surface += statement_wrapper(
    res[pydash.get(my_birth, "id")],
    ont.EventDate,
    rdflib.Literal(pydash.get(my_birth, "event_date")),
    pydash.get(my_birth, "visible"),
    key_chain,
)
surface += statement_wrapper(
    res[pydash.get(my_birth, "id")],
    ont.EventLocation,
    res[pydash.get(my_birth, "event_location.id")],
    pydash.get(my_birth, "visible"),
    key_chain,
)
surface += statement_wrapper(
    res[pydash.get(my_birth, "event_location.id")],
    rdflib.RDFS.label,
    rdflib.Literal(pydash.get(my_birth, "event_location.name")),
    pydash.get(my_birth, "visible"),
    key_chain,
)

for x in pydash.get(my_birth, "agents"):
    activity_id = rdflib.BNode().skolemize()
    surface += statement_wrapper(
        res[pydash.get(my_birth, "id")],
        ont.hasActivity,
        activity_id,
        pydash.get(x, "visible"),
        key_chain,
    )
    surface += statement_wrapper(
        activity_id,
        rdflib.RDF.type,
        ont[pydash.get(x, "activity")],
        pydash.get(x, "visible"),
        key_chain,
    )
    surface += statement_wrapper(
        activity_id,
        ont.hasAgent,
        res[pydash.get(x, "id")],
        pydash.get(x, "visible"),
        key_chain,
    )
    surface += statement_wrapper(
        res[pydash.get(x, "id")],
        rdflib.RDFS.label,
        rdflib.Literal(pydash.get(x, "name")),
        pydash.get(x, "visible"),
        key_chain,
    )
    surface += statement_wrapper(
        res[pydash.get(x, "id")],
        rdflib.RDF.type,
        ont.Person,
        pydash.get(x, "visible"),
        key_chain,
    )

key_chain_path = pathlib.Path.cwd().parents[0] / 'authentication' / 'keys.json'

if not key_chain_path.exists():
    print('created it')
    with open(key_chain_path, "w") as dump_keys:
        json.dump(key_chain, dump_keys, indent=4)
else:
    with open(key_chain_path) as load_keys:
        key_data = json.load(load_keys)

        for a,b in key_chain.items():
            key_data[a] = b

        with open(key_chain_path, "w") as dump_keys:
            json.dump(key_data, dump_keys, indent=4)

print(len(surface), "triples.")


In [ ]:
# encrypt triples on closed statements.

graph_out = rdflib.Graph()

closed = [
    s for s, p, o in surface.triples((None, rdflib.RDF.type, ont.ClosedStatement))
]

for s, p, o in surface.triples((None, None, None)):
    if s in closed and p == ont.hasPayload:
        enc_statement = ansible_vault.Vault(key_chain[str(s)]).dump_raw(o)
        graph_out.add((s, p, rdflib.Literal(enc_statement)))
    else:
        graph_out.add((s, p, o))

timestamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
graph_out.serialize(destination=str(pathlib.Path.cwd().parents[0] / 'rdf' / f'{timestamp}.json'), format="json-ld")
print(len(graph_out), "triples.")


In [ ]:
# # parse open statements.

# graph_in = rdflib.Graph()
# graph_in.parse(pathlib.Path.cwd() / "data.json")

# openstate = [
#     s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.OpenStatement))
# ]

# graph_out = rdflib.Graph()
# graph_out.bind("ont", ont)
# graph_out.bind("res", res)

# for s, p, o in graph_in.triples((None, None, None)):
#     if s in openstate and p == ont.hasPayload:
#         fragment = rdflib.Graph()
#         fragment.parse(data=o)
#         graph_out += fragment

# print(graph_out.serialize(format="ttl"))


In [ ]:
# # parse closed statements (where access has been granted).

# subject = res["94efddce-3f2a-4838-a5fa-fcf4180e5584"]
# subject_matter = list()

# with open(pathlib.Path.cwd().parents[0] / "example" / "keys.json") as keys:
#     keys = json.load(keys)

# closedstate = [
#     s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.ClosedStatement))
# ]

# for s, p, o in graph_in.triples((None, None, None)):
#     if s in closedstate and p == ont.hasPayload:
#         fragment = rdflib.Graph()
#         unencruptr = ansible_vault.Vault(keys[str(s)]).load_raw(o).decode()
#         fragment.parse(data=unencruptr)
#         for a, b, c in fragment.triples((None, None, None)):
#             if subject in [a, b, c]:
#                 subject_matter.append(str(s))

# exposed_keys = {a: b for a, b in keys.items() if str(a) in subject_matter}

# graph_in = rdflib.Graph()
# graph_in.parse(pathlib.Path.cwd() / "data.json")

# graph_out = rdflib.Graph()
# graph_out.bind("ont", ont)
# graph_out.bind("res", res)

# for s, p, o in graph_in.triples((None, None, None)):
#     if str(s) in exposed_keys.keys() and p == ont.hasPayload:
#         fragment = rdflib.Graph()
#         unencruptr = ansible_vault.Vault(exposed_keys[str(s)]).load_raw(o).decode()
#         fragment.parse(data=unencruptr)
#         graph_out += fragment

# print(graph_out.serialize(format="ttl"))
